In [1]:
import time
import numpy as np
import gzip
import tensorflow as tf
import pandas as pd

import data_access as data

from tqdm import tqdm

In [2]:
reader = data.load_deals(gzip.open('../testdata.gz'))

In [3]:
X, contracts = next(reader)

In [4]:
X[0,:,:,3]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]])

In [5]:
X[0,:,:,3].reshape((1, 52))

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,
         0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]])

In [6]:
n_bids = len(data.bid2id)

In [7]:
num_steps = 32
lstm_size = 128

lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)

x_in = tf.placeholder(tf.float32, [1, 2 + 3*n_bids + 52], 'x_in')

state_c = tf.placeholder(tf.float32, [1, lstm.state_size.c], 'state_c')
state_h = tf.placeholder(tf.float32, [1, lstm.state_size.h], 'state_h')

init_c = np.zeros((1, lstm.state_size.c))
init_h = np.zeros((1, lstm.state_size.h))

softmax_w = tf.get_variable('softmax_w', shape=[lstm.output_size, n_bids], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
softmax_b = tf.Variable(np.zeros((1, n_bids)), dtype=tf.float32)


In [8]:
sess = tf.InteractiveSession()

In [9]:
def encode_bid(bid):
    bid_one_hot = np.zeros((1, n_bids), dtype=np.float32)
    bid_one_hot[0, data.bid2id[bid]] = 1
    return bid_one_hot

def get_input(lho_bid, partner_bid, rho_bid, hand, v_we, v_them):
    vuln = np.array([[v_we, v_them]], dtype=np.float32)
    return np.concatenate((vuln, encode_bid(lho_bid), encode_bid(partner_bid), encode_bid(rho_bid), hand), axis=1)

In [10]:
x_in_0 = get_input('PAD_START', 'PAD_START', 'PAD_START', X[0,:,:,0].reshape((1, 52)), False, False)

In [11]:
output, (next_c, next_h) = lstm(x_in, (state_c, state_h))

In [12]:
output.shape

TensorShape([Dimension(1), Dimension(128)])

In [13]:
out_bid = tf.nn.softmax(tf.add(tf.matmul(output, softmax_w), softmax_b))

In [14]:
out_bid.shape

TensorShape([Dimension(1), Dimension(40)])

In [15]:
sess.run(tf.global_variables_initializer())

In [16]:
%time
out_bid_0 = sess.run(out_bid, feed_dict={x_in: x_in_0, state_c: init_c, state_h: init_h})

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs


In [17]:
%time
sess.run(next_c, feed_dict={x_in: x_in_0, state_c: init_c, state_h: init_h})

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs


array([[ 0.03513054, -0.14355598, -0.04834708, -0.25853854,  0.08921582,
        -0.03133602, -0.17231166, -0.1114969 ,  0.0155274 , -0.12208191,
         0.02189039,  0.13560772,  0.23195234,  0.06138988,  0.16721036,
         0.05136308,  0.03310371, -0.04438537,  0.0465861 ,  0.12211782,
        -0.01246611,  0.05344652,  0.0482442 ,  0.08670079, -0.01844062,
         0.02018764, -0.02305588, -0.06913504, -0.01867585,  0.09721985,
        -0.00559311, -0.12044452, -0.0604309 , -0.15839787, -0.08757549,
        -0.00179401,  0.12353435, -0.01312838,  0.0290947 , -0.11663657,
         0.10123291,  0.07003731, -0.1912934 , -0.05704527,  0.07295059,
        -0.04472804, -0.14155932, -0.0686332 ,  0.14684923,  0.07269339,
        -0.09894346, -0.01096533, -0.00671386,  0.16663069, -0.13340357,
         0.12920776,  0.12024439,  0.05776527,  0.03414896, -0.05918268,
         0.06411506, -0.20459491,  0.01481543, -0.05068365, -0.11199539,
        -0.12375126,  0.0349847 , -0.0434694 ,  0.0

In [18]:
%time
sess.run(next_h, feed_dict={x_in: x_in_0, state_c: init_c, state_h: init_h})

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


array([[ 0.01609665, -0.06077044, -0.0244471 , -0.11951753,  0.04720319,
        -0.01583029, -0.09008619, -0.05796791,  0.00710448, -0.06521562,
         0.01134629,  0.07159963,  0.12322612,  0.02933597,  0.08318885,
         0.02335306,  0.01485762, -0.02142076,  0.02155773,  0.0500484 ,
        -0.00569134,  0.02838464,  0.02571752,  0.04851871, -0.00996098,
         0.00952325, -0.01223299, -0.03512239, -0.00959018,  0.04428341,
        -0.00294821, -0.04804185, -0.02793876, -0.08189852, -0.0399058 ,
        -0.00098585,  0.06023633, -0.00734918,  0.01595885, -0.04753117,
         0.05711212,  0.04230979, -0.0900789 , -0.02749023,  0.03292326,
        -0.02333415, -0.07432293, -0.03431798,  0.0708868 ,  0.04081799,
        -0.05996556, -0.00462106, -0.00383686,  0.08142198, -0.0602603 ,
         0.07056093,  0.06388619,  0.02556089,  0.01600537, -0.0260678 ,
         0.03419851, -0.0836817 ,  0.00904017, -0.0261287 , -0.06259164,
        -0.05950082,  0.02166935, -0.02150986,  0.0

In [19]:
out_bid_0

array([[ 0.02403424,  0.02583252,  0.02387475,  0.02665216,  0.02560438,
         0.02573131,  0.02635817,  0.02497757,  0.02397253,  0.02405758,
         0.02471473,  0.02511335,  0.02500698,  0.02724934,  0.02502115,
         0.02674881,  0.02768386,  0.02175892,  0.02438564,  0.02438872,
         0.02244969,  0.02404417,  0.02820112,  0.02514033,  0.02555827,
         0.02259923,  0.02541686,  0.02523277,  0.02372505,  0.02530306,
         0.02331924,  0.02489634,  0.0217726 ,  0.02369095,  0.02368653,
         0.02635667,  0.02726492,  0.02482493,  0.02679865,  0.02655185]], dtype=float32)

In [137]:
def is_contract(bid):
    return bid[0].isdigit()

def can_double(auction):
    if len(auction) == 0:
        return False
    if is_contract(auction[-1]):
        return True
    if len(auction) >= 3 and is_contract(auction[-3]) and auction[-2] == 'PASS' and auction[-1] == 'PASS':
        return True
    return False

def can_redouble(auction):
    if len(auction) == 0:
        return False
    if auction[-1] == 'X':
        return True
    if len(auction) >= 3 and auction[-3] == 'X' and auction[-2] == 'PASS' and auction[-1] == 'PASS':
        return True
    return False

def last_contract(auction):
    for bid in reversed(auction):
        if is_contract(bid):
            return bid
    return None

def contract_level_step(contract):
    return int(contract[0])*5 + data.suit_rank[contract[1]]

def is_higher_contract(this_contract, other_contract):
    return contract_level_step(this_contract) > contract_level_step(other_contract)

def can_bid_contract(bid, auction):
    assert is_contract(bid)
    contract = last_contract(auction)
    if contract is None:
        return True
    return is_higher_contract(bid, contract)

def auction_over(auction):
    if len(auction) < 4:
        return False
    if auction[-1] == 'PAD_END':
        return True
    contract = last_contract(auction)
    if contract is None:
        return all([bid == 'PASS' for bid in auction[-4:]]) and all([bid == 'PAD_START' for bid in auction[:-4]])
    else:
        return all([bid == 'PASS' for bid in auction[-3:]])

def can_bid(bid, auction):
    if bid == 'PASS':
        return True
    if bid == 'X':
        return can_double(auction)
    if bid == 'XX':
        return can_redouble(auction)
    if is_contract(bid):
        return can_bid_contract(bid, auction)
    return False

def sample_bid(auction, from_bids):
    if auction_over(auction):
        return 'PAD_END'
    while True:
        bid_one_hot = np.random.multinomial(1, from_bids[0])
        bid_id = np.argmax(bid_one_hot)
        bid = data.id2bid[bid_id]
        if can_bid(bid, auction):
            return bid
        
def get_contract(auction):
    contract = None
    doubled = False
    redoubled = False
    last_bid_i = None
    for i in reversed(range(len(auction))):
        bid = auction[i]
        if is_contract(bid):
            contract = bid
            last_bid_i = i
            break
        if bid == 'X':
            doubled = True
        if bid == 'XX':
            redoubled = True
    
    if contract is None:
        return None
    
    declarer_i = None
    for i in range(last_bid_i + 1):
        bid = auction[i]
        if not is_contract(bid):
            continue
        if (i + last_bid_i) % 2 != 0:
            continue
        if bid[1] != contract[1]:
            continue
        declarer_i = i
        break
        
    declarer = ['N', 'E', 'S', 'W'][declarer_i % 4]
    
    xx = '' if not doubled else 'X' if not redoubled else 'XX'
    
    return contract + xx + declarer

In [138]:
def get_par(contracts, vuln_ns, vuln_ew):
    side_vuln = [int(vuln_ns), int(vuln_ew)]
    side = {'N': 0, 'E': 1, 'S': 0, 'W': 1}
    
    contract_scores = sorted(contracts.items(), key=lambda cs: (int(cs[0][0]) * 5 + data.suit_rank[cs[0][1]], cs[0]))
    
    best_score = [0, 0]
    best_contract = [None, None]
    
    for contract, scores in contract_scores:
        side_i = side[contract[-1]]
        score = scores[side_vuln[side_i]]
        
        if score > best_score[side_i]:
            if score > 0 and 'X' in contract:
                continue
            if score < 0 and 'X' not in contract:
                continue
            best_score[side_i] = score
            best_score[(side_i + 1) % 2] = -score
            best_contract[side_i] = contract
            best_contract[(side_i + 1) % 2] = contract
            
    assert best_contract[0] == best_contract[1]
            
    return best_contract[0]

In [139]:
is_contract('4S'), is_contract('PASS'), is_contract('X')

(True, False, False)

In [140]:
can_double(['PASS', '1S', 'X', 'PASS'])

False

In [141]:
last_contract(['PASS', '1S', 'X', 'PASS'])

'1S'

In [142]:
auction = ['PAD_START']
while not auction_over(auction):
    auction.append(sample_bid(auction, out_bid_0))
print(auction)
print(get_contract(auction))

['PAD_START', '1H', '5N', '7S', 'PASS', '7N', 'X', 'XX', 'PASS', 'PASS', 'PASS']
7NXXE


In [143]:
get_contract(['1S', '1N', 'PASS', '2H', 'PASS', '2S', 'PASS', '4S', 'PASS', 'PASS', 'PASS'])

'4SE'

In [180]:
class Simulator(object):
    
    def __init__(self, deal, contracts, model, sess):
        self.deal = deal
        self.contracts = contracts
        self.hands = [
            deal[0,:,:,0].reshape((1, 52)), 
            deal[0,:,:,1].reshape((1, 52)),
            deal[0,:,:,2].reshape((1, 52)),
            deal[0,:,:,3].reshape((1, 52)),
        ]
        self.model = model
        self.sess = sess
        
        self.par = get_par(self.contracts, False, False)
        
        self.cache = {}
        
    def simulate_bid(self, auction, s_c, s_h, n=1):
        i = len(auction) % 4
        padded_auction = (['PAD_START'] * max(0, 3 - len(auction))) + auction
        auction_key = tuple(auction)
        if auction_key not in self.cache:
            x_input = get_input(padded_auction[-3], padded_auction[-2], padded_auction[-1], self.hands[i], False, False)
            out_bid_np, next_c_np, next_h_np = sess.run([out_bid, next_c, next_h], feed_dict={x_in: x_input, state_c: s_c, state_h: s_h})
            self.cache[auction_key] = (out_bid_np, next_c_np, next_h_np)
        else:
            out_bid_np, next_c_np, next_h_np = self.cache[auction_key]
        bids = []
        last_contract = get_contract(padded_auction)
        while len(bids) < n:
            s_bid = sample_bid(padded_auction, out_bid_np)
            if is_contract(s_bid) and is_higher_contract(s_bid, self.par):
                 continue
            if 'X' in s_bid and contract_level_step(last_contract) == contract_level_step(self.par) and last_contract[-1] == self.par[-1]:
                if s_bid not in self.par:
                    continue
            bids.append(s_bid)
        return bids, (next_c_np, next_h_np)
    
    def simulate_auction(self, auction, s_c, s_h):
        sim_auction = auction[:]
        C, H = s_c, s_h
        while not auction_over(sim_auction):
            bids, (next_c_np, next_h_np) = self.simulate_bid(sim_auction, C, H, 1)
            sim_auction.append(bids[0])
            C = next_c_np
            H = next_h_np
        return sim_auction
    
    def best_bid(self, auction, s_c, s_h, n=100):
        results = {}
        declarer2i = {seat:i for i, seat in enumerate(['N', 'E', 'S', 'W'])}
        bids, (next_c_np, next_h_np) = self.simulate_bid(auction, s_c, s_h, n)
        for bid in bids:
            sim_auction = self.simulate_auction(auction + [bid], next_c_np, next_h_np)
            sim_contract = get_contract(sim_auction)
            if sim_contract is not None:
                seat_to_bid = len(auction) % 4
                declarer_seat = declarer2i[sim_contract[-1]]
                sign = 1 if (seat_to_bid + declarer_seat) % 2 == 0 else -1
                score = sign * self.contracts.get(sim_contract, (0, 0))[0]
            else:
                score = 0
            if bid not in results:
                results[bid] = [0, 0]
            results[bid][0] += score
            results[bid][1] += 1
        max_score_bid = max(((v[0] / v[1], k) for k, v in results.items()))
        return max_score_bid
    
    def best_auction(self, auction, s_c, s_h, n=100):
        self.cache = {}
        best_auction = auction[:]
        while not auction_over(best_auction):
            score, bid = self.best_bid(best_auction, s_c, s_h, n)
            best_auction.append(bid)
        return score, best_auction
    

In [145]:
type(next_c)

tensorflow.python.framework.ops.Tensor

In [146]:
simulator = Simulator(X, contracts, lstm, sess)

bid, (next_c_np, next_h_np) = simulator.simulate_bid(['6N', 'PASS'], init_c, init_h)

In [147]:
bid

['PASS']

In [86]:
simulator.cache = {}

In [87]:
%time simulator.best_bid(['7H', 'PASS'], init_c, init_h, 20)

CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 96.5 ms


(-1115.0, 'PASS')

In [116]:
%time simulator.best_auction([], init_c, init_h, 10)

CPU times: user 352 ms, sys: 4 ms, total: 356 ms
Wall time: 519 ms


(-2900.0, ['5H', '6N', 'X', 'PASS', 'PASS', 'PASS'])

In [92]:
simulator.par

'6NN'

In [93]:
len(simulator.cache)

2874

In [94]:
get_contract(['7S', 'PASS', 'PASS', 'PASS'])

'7SN'

In [95]:
contracts['7SN']

(-50, -100)

In [96]:
get_par(contracts, False, False)

'6NN'

In [168]:
X2, contracts2 = next(reader)

In [169]:
get_par(contracts2, False, False)

'4SN'

In [170]:
%%timeit
get_par(contracts, True, False)

1.18 ms ± 365 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [185]:
sim2 = Simulator(X2, contracts2, lstm, sess)

In [186]:
sim2.par

'4SN'

In [217]:
%time sim2.best_auction([], init_c, init_h, 10)

CPU times: user 1.22 s, sys: 20 ms, total: 1.24 s
Wall time: 1.49 s


(100.0,
 ['1S',
  '2C',
  '2H',
  'X',
  '3C',
  '3H',
  '3N',
  'X',
  '4C',
  'PASS',
  '4H',
  'PASS',
  'PASS',
  'PASS'])

In [212]:
sim2.par

'4SN'

In [110]:
get_contract(['1D', '1H', '1S', 'X', 'PASS', '1N', 'PASS', 'PASS', 'PASS'])

'1NE'

In [214]:
contracts2['4DS']

(-250, -500)

In [218]:
a = np.array([np.array([1, 2]), np.array([1, 2, 3])])

In [219]:
a

array([array([1, 2]), array([1, 2, 3])], dtype=object)

In [220]:
a.shape

(2,)

In [221]:
13 / 4

3.25

In [224]:
(8 - 3.25)/1.75

2.7142857142857144

In [225]:
tf.contrib.rnn.MultiRNNCell?